## Visualisation des différentes sources de données 

L'objectif de ce notebook est de coupler les données récupéré au travers du processing des différentes souces de données dans les dossiers correspondants 

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from cartiflette import carti_download
import matplotlib.patches as mpatches

#Récupération des geodataframes pour tracer la carte 
gdf_nv_risque = gpd.read_file("/home/onyxia/work/Python-For-Data-Science-Project/TRI Shapefile Processing/gdf_nv_risque.geojson")
gdf_type_risque = risk_data = pd.read_csv("/home/onyxia/work/Python-For-Data-Science-Project/TRI API Processing /Type_inondation_data.csv")
gdf_cours_eau = gpd.read_file("/home/onyxia/work/Python-For-Data-Science-Project/TRI Shapefile Processing/gdf_cours_eau.geojson")

# Convertir les colonnes de type Timestamp en chaînes de caractères dans gdf_nv_risque
for col in gdf_nv_risque.columns:
    if gdf_nv_risque[col].dtype == 'datetime64[ms]':
        gdf_nv_risque[col] = gdf_nv_risque[col].astype(str)


# Charger les données Cartiflette pour les communes du Gard
Gard = carti_download(
    crs=4326,
    values="30",
    borders="COMMUNE",
    vectorfile_format="geojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

# Charger les informations sur la nature des risques
gdf_type_risque_grouped = gdf_type_risque.groupby('code_insee')['libelle_risque_long'].apply(
    lambda x: ', '.join(x.unique())
).reset_index()
Gard_type_risque = Gard.merge(gdf_type_risque_grouped, left_on='INSEE_COM', right_on='code_insee', how='left')
Gard_type_risque['libelle_risque_long'] = Gard_type_risque['libelle_risque_long'].fillna("Pas de risque identifié")


In [ ]:
import folium
from folium.features import GeoJson, GeoJsonTooltip

# Créer une carte Folium centrée sur le Gard
m = folium.Map(location=[44.0, 4.0], zoom_start=9, tiles="cartodbpositron")

# Ajouter les contours des communes et la nature des risques avec `Gard_with_risks`
geojson_communes = folium.GeoJson(
    Gard_type_risque,
    style_function=lambda x: {"fillColor": "transparent", "color": "black", "weight": 1, "fillOpacity": 0},
    highlight_function=lambda x: {"fillColor": "yellow", "color": "black", "weight": 2, "fillOpacity": 0.3},
    tooltip=GeoJsonTooltip(
        fields=["libelle_risque_long"],  # Afficher la nature des risques
        aliases=["Nature du risque :"],  # Texte dans l'info-bulle
        localize=True
    )
)
geojson_communes.add_to(m)

# Sauvegarder et afficher la carte
m.save("carte_communes_risques.html")
m


In [ ]:
import folium
from folium.features import GeoJson, GeoJsonTooltip

# Créer une carte Folium centrée sur le Gard
m = folium.Map(location=[44.0, 4.0], zoom_start=9, tiles="cartodbpositron")

# Étape 1 : Ajouter les contours des communes et la nature des risques avec `Gard_with_risks`
geojson_communes = folium.GeoJson(
    Gard_type_risque,
    style_function=lambda x: {"fillColor": "transparent", "color": "black", "weight": 1, "fillOpacity": 0},
    highlight_function=lambda x: {"fillColor": "yellow", "color": "black", "weight": 2, "fillOpacity": 0.3},
    tooltip=GeoJsonTooltip(
        fields=["libelle_risque_long"],  # Afficher la nature des risques
        aliases=["Nature du risque :"],  # Texte dans l'info-bulle
        localize=True
    )
)
geojson_communes.add_to(m)

# Étape 2 : Ajouter les cours d’eau dans le Gard avec `waterways_clipped`
geojson_waterways = folium.GeoJson(
    gdf_cours_eau,
    style_function=lambda x: {"color": "blue", "weight": 1.5, "fillOpacity": 0.7},  # Style des cours d'eau
    tooltip=GeoJsonTooltip(fields=[], aliases=[])  # Pas d'info-bulle pour les cours d'eau
)
geojson_waterways.add_to(m)

#afficher la carte

m


In [ ]:
import folium
from folium.features import GeoJson, GeoJsonTooltip

# Créer une carte Folium centrée sur le Gard
m = folium.Map(location=[44.0, 4.0], zoom_start=9, tiles="cartodbpositron")

# Étape 1 : Ajouter les polygones des niveaux de risque en arrière-plan (`gdf_nv_risque`)
def style_function_risk(feature):
    risk_level = feature['properties'].get('typ_inond1', None)
    if risk_level == 1:  # Risque faible
        color = "lightblue"
    elif risk_level == 3:  # Risque élevé
        color = "lightcoral"
    else:
        color = "gray"  # Pas de données
    return {"fillColor": color, "color": "black", "weight": 0.5, "fillOpacity": 0.6}

geojson_risks = folium.GeoJson(
    gdf_nv_risque,
    style_function=style_function_risk  # Appliquer les couleurs basées sur `typ_inond1`
)
geojson_risks.add_to(m)

# Étape 2 : Ajouter les contours des communes avec surlignage et info-bulle (`Gard_with_risks`)
geojson_communes = folium.GeoJson(
    Gard_type_risque,
    style_function=lambda x: {"fillColor": "transparent", "color": "black", "weight": 1, "fillOpacity": 0},
    highlight_function=lambda x: {"fillColor": "yellow", "color": "black", "weight": 2, "fillOpacity": 0.3},
    tooltip=GeoJsonTooltip(
        fields=["libelle_risque_long"],  # Afficher la nature des risques
        aliases=["Nature du risque :"],  # Texte dans l'info-bulle
        localize=True
    )
)
geojson_communes.add_to(m)

# Étape 3 : Ajouter les cours d’eau dans le Gard avec `waterways_clipped`
geojson_waterways = folium.GeoJson(
    gdf_cours_eau,
    style_function=lambda x: {"color": "blue", "weight": 1.5, "fillOpacity": 0.7}
)
geojson_waterways.add_to(m)

# Étape 4 : Ajouter une légende
legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 250px;
    background-color: white;
    z-index:9999;
    padding: 10px;
    border: 1px solid gray;
">
    <h4>Légende</h4>
    <i style="background: lightblue; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Risque faible<br>
    <i style="background: lightgreen; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Risque modéré<br>
    <i style="background: lightcoral; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Risque élevé<br>
    <i style="border: 1px solid black; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Bordure des communes<br>
    <i style="background: blue; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Cours d'eau<br>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Sauvegarder et afficher la carte
m.save("carte_risques_nature_legende.html")
m


In [ ]:
gdf_nv_risque_projected = gdf_nv_risque.to_crs(epsg=2154)  # CRS projeté
Gard_type_risque_projected = Gard_type_risque.to_crs(epsg=2154)

#Filtrer les zones avec une nature de risque déterminée
Gard_with_valid_risks = Gard_type_risque_projected[Gard_type_risque_projected['libelle_risque_long'] != "Pas de risque identifié"]

#la fonction area permet de déterminer la superficie totale des zones à risque d'inondation dans le Gard. 
gdf_nv_risque_projected['area'] = gdf_nv_risque_projected.geometry.area
total_area = gdf_nv_risque_projected['area'].sum()

#Jointure spaciale afin de déterminer les zones de chevauchement entre nature et niveau de risque 
intersections = gpd.overlay(Gard_with_valid_risks, gdf_nv_risque_projected, how="intersection")

#déterminer la superficie totale des intersections
intersections['area'] = intersections.geometry.area
intersected_area = intersections['area'].sum()

#Finalement, nous calculons le le pourcentage
percentage_intersection = (intersected_area / total_area) * 100

print(f"Le Pourcentage d'intersection entre territoire avec un niveau de risque supérieur ou égal à un et un libellé de risque est de {percentage_intersection:.2f}%")


In [ ]:
import folium
from folium.features import GeoJson, GeoJsonTooltip
from folium.plugins import MarkerCluster
import geopandas as gpd

gdf_trans_immo = gpd.read_file('/home/onyxia/work/Python-For-Data-Science-Project/DVF Processing/gdf_trans_immo.geojson')

# Créer une carte Folium centrée sur le Gard
m = folium.Map(location=[44.0, 4.0], zoom_start=9, tiles="cartodbpositron")

# Étape 1 : Ajouter les polygones des niveaux de risque (`gdf_nv_risque`)
def style_function_risk(feature):
    risk_level = feature['properties'].get('typ_inond1', None)
    if risk_level == 1:  # Risque faible
        color = "lightblue"
    elif risk_level == 3:  # Risque élevé
        color = "lightcoral"
    else:
        color = "gray"  # Pas de données
    return {"fillColor": color, "color": "black", "weight": 0.5, "fillOpacity": 0.6}

geojson_risks = folium.GeoJson(
    gdf_nv_risque,
    style_function=style_function_risk
)
geojson_risks.add_to(m)

# Étape 2 : Ajouter les contours des communes (`Gard_with_risks`)
geojson_communes = folium.GeoJson(
    Gard_type_risque,
    style_function=lambda x: {"fillColor": "transparent", "color": "black", "weight": 1, "fillOpacity": 0},
    )

geojson_communes.add_to(m)

# Étape 3 : Ajouter les cours d’eau dans le Gard (`waterways_clipped`)
geojson_waterways = folium.GeoJson(
    gdf_cours_eau,
    style_function=lambda x: {"color": "blue", "weight": 1.5, "fillOpacity": 0.7}
)
geojson_waterways.add_to(m)

# Étape 4 : Ajouter les points des transactions immobilières (`gdf_trans_immo`) avec un MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

for _, row in gdf_trans_immo.iterrows():
    # Vérifier que la géométrie est un Point
    if row.geometry.geom_type == "Point":
        # Extraire les informations pour la tooltip
        valeurfonc = row.get("properties.valeurfonc", "Non disponible")
        datemut = row.get("properties.datemut", "Non disponible")
        sbati = row.get("properties.sbati", "Non disponible")
        libtypbien = row.get("properties.libtypbien", "Non disponible")

        # Ajouter le marqueur avec la tooltip
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            tooltip=(
                f"<b>Valeur foncière :</b> {valeurfonc}<br>"
                f"<b>Date de transaction :</b> {datemut}<br>"
                f"<b>Nombre de m² :</b> {sbati}<br>"
                f"<b>Type de propriété :</b> {libtypbien}"
            )
        ).add_to(marker_cluster)

# Étape 5 : Ajouter une légende
legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 250px;
    background-color: white;
    z-index:9999;
    padding: 10px;
    border: 1px solid gray;
">
    <h4>Légende</h4>
    <i style="background: lightblue; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Risque faible<br>
    <i style="background: lightcoral; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Risque élevé<br>
    <i style="border: 1px solid black; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Bordure des communes<br>
    <i style="background: blue; width: 15px; height: 15px; float: left; margin-right: 5px;"></i> Cours d'eau<br>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Sauvegarder et afficher la carte
m.save("carte_transaction_risque.html")
print("Carte combinée générée et sauvegardée dans 'carte_transaction_risque.html'.")


In [ ]:
m

# Modélisation quantitative

Cette section vient compléter l'analyse exploratoire menée plus haut pour essayer d'offrir une modeste quantification de la relation entre risque d'inondations et prix de l'immobilier, sans essayer d'obtenir une interprétation qui soit causale. 

Pour ce faire, nous nous reposons sur un regroupement de données cross-sectionnelles (*pooled cross-sectional* en anglais) que nous avons pu assembler à partir des bases de données DVF et Géorisque. 
Ces données comprennent notamment les variables suivantes:
- La valeur à laquelle s'est déroulée la transaction immobilière, que nous ajusterons de l'inflation.
- Le niveau de risque inhérent à la position géographique du bien, qui est encodé entre trois niveaux de risques: faible, moyen et élevé.
- La surface en mètre carrée du bien.

Nous estimerons ainsi l'équation économétrique suivante:

$ Prix = \beta_0 + \beta_1*R_{élevé} + \beta_2*R_{moyen} + \beta_3*Surface + \epsilon $

Avec:
- $Prix$, la valeur de la transaction immobilière en euro constant de 2015. Nous faisons cet ajustement pour isoler l'effet de l'inflation et donc mieux capturer les interactions liées aux risques d'inondation. De plus, nous avons exclu de la base de données les observations où les valeurs de transactions étaient supérieures au 95ème quantile ou inférieure au 5ème quantile. Ceci permet de se séparer des valeurs aberrantes qui pourraient avoir un effet disproportionné sur nos coefficients de régression.
- $R_{élevé}$ et $R_{moyen}$ sont nos variables binaires (*dummies*) liés au niveau de risque du bien immobilier. Le niveau de référence pour de ces *dummies* est le risque de niveau faible.
- Et pour finir, $Surface$ correspond à la surface en mètre carré du bien immobilier. 

Nous présentons ci-dessous les résultats de notre estimation par la méthode des moindres carrés.

Sans surprise, nous observons avec satisfaction que ce modèle économétrique permet de capturer les relations clefs. En effet, le signe des coefficients de régression correspond parfaitement à nos attentes. Nous constatons qu'un bien immobilier se trouvant dans une zone à haut risque voit son prix de transaction baisser de [INSERER LE COEFF] en moyenne, *ceteris paribus*. L'effet est plus modeste en taille mais similaire en direction pour les biens situées en zone à risque modéré. De plus, les deux coefficients sont statistiquement significatifs.

Cependant, il ne nous est pas étranger que notre modèle souffre, entre autres, du biais de variables omises. En effet, il y a bien d'autres variables pouvant impacter le prix d'un bien immobilier que celles que nous avons inclus: par exemple, la distance à des moyens de transports, la distance au centre-ville le plus proche, et même la distance au littoral le plus proche, en particulier dans le sud de la France. 